In [1]:
import pandas as pd
import numpy as np
import pickle
import Normalization
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [3]:
# Df : FPKM values of each sample
# Driver_Mutation_Matrix : Answer label of each sample
# gene_N : DataFrame representing up to 100 genes that interact with the each target driver gene 
Df=pd.read_csv("/data2/sano/TCGA_Version_LAST/TCGA_RNA_common_patient.csv", index_col=0)
Df=Df.T

Driver_Mutation_Matrix=pd.read_csv("/data2/sano/TCGA_Version_LAST/TCGA_Answer_common_patient.csv", index_col=0)
gene_id=pd.read_table("/data2/sano/TCGA_Version_LAST/Gene_id.txt", sep="\t", index_col=0)
Gene_N=pd.read_csv( "/data2/sano/TCGA_Version_LAST/Biogrid_Gene_100_gene.csv", index_col=0)

## Train_Test_Patients['train']: list of the patient name of trainig dataset
## Train_Test_Patients['test']: list of the patient name of trainig dataset
## Train_Test_Patients['test_tumor_type']: list of tumor types of trainig dataset patients
## Train_Test_Patients['train_tumor_type']: list of tumor types of test dataset patients
filename = "/data2/sano/TCGA_Version_LAST/Train_Test_patient.pickle"
with open(filename, 'rb') as f:
    Train_Test_Patients=pickle.load(f)

In [16]:
Df.head()

,TCGA-26-5134-01,TCGA-ER-A2NC-06,TCGA-68-A59J-01,TCGA-AN-A0XW-01,TCGA-A2-A0EO-01,TCGA-AG-3885-01,TCGA-DX-A7EF-01,TCGA-EL-A3CT-01,TCGA-A2-A0ER-01,TCGA-A8-A09M-01,...,TCGA-WT-AB41-01,TCGA-22-4601-01,TCGA-49-6743-01,TCGA-UF-A71A-01,TCGA-AB-2811-03,TCGA-IB-AAUV-01,TCGA-ZG-A8QZ-01,TCGA-D3-A51N-06,TCGA-BQ-5890-01,TCGA-KO-8416-01
ENSG00000242268,1.518129,0.425279,0.048830,0.147290,0.000000,0.000000,0.000000,0.214315,0.000000,0.024092,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.046110,0.000000,1.523623,0.076606,0.000000
ENSG00000270112,0.737258,0.068326,0.000000,0.003406,0.000000,0.000000,0.003597,0.000000,0.000000,0.000000,...,0.000000,0.007313,0.004925,0.000000,0.000000,0.006398,0.000000,0.082725,0.002657,0.847032
ENSG00000167578,7.916805,2.700470,5.494976,4.874187,1.041235,4.720237,1.187594,1.917023,1.600615,4.627177,...,16.956701,2.765771,2.086688,3.502492,7.094663,3.317904,2.461776,16.772956,2.828654,3.761424
ENSG00000273842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000078237,6.507165,5.814275,6.869661,6.440140,3.377336,1.388719,2.514847,2.723141,2.782282,1.358016,...,1.907473,3.181146,4.368159,5.130821,1.766985,3.514252,2.398306,4.900951,1.898173,2.758433


In [15]:
Driver_Mutation_Matrix.head()

,FBXW7,ARID2,WHSC1,ALK,RUNX1,AXIN1,ERBB2,MET,KDR,EZH2,...,DDX3X,CDK12,RNF43,NF2,SETDB1,NFE2L2,TCF7L2,SMAD4,MAX,MEN1
TCGA-26-5134-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-ER-A2NC-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-68-A59J-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
TCGA-AN-A0XW-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
TCGA-A2-A0EO-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
gene_id.head()

,Previous symbols,Alias symbols,Ensembl ID(supplied by Ensembl)
Approved symbol,,,
A1BG,NaN,NaN,ENSG00000121410
A1BG-AS1,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569,ENSG00000268895
A1CF,NaN,"ACF, ASP, ACF64, ACF65, APOBEC1CF",ENSG00000148584
A1S9T,NaN,NaN,NaN
A2M,NaN,"FWP007, S863-7, CPAMD5",ENSG00000175899


In [12]:
Gene_N.head()

,FBXW7,ARID2,WHSC1,ALK,RUNX1,AXIN1,ERBB2,MET,KDR,EZH2,...,DDX3X,CDK12,RNF43,NF2,SETDB1,NFE2L2,TCF7L2,SMAD4,MAX,MEN1
0,ENSG00000109670,ENSG00000189079,ENSG00000109685,ENSG00000171094,ENSG00000159216,ENSG00000103126,ENSG00000141736,ENSG00000105976,ENSG00000128052,ENSG00000106462,...,ENSG00000215301,ENSG00000167258,ENSG00000108375,ENSG00000186575,ENSG00000143379,ENSG00000116044,ENSG00000148737,ENSG00000141646,ENSG00000125952,ENSG00000133895
1,ENSG00000113558,ENSG00000073584,ENSG00000116478,ENSG00000124181,ENSG00000067955,ENSG00000082701,ENSG00000146648,ENSG00000110395,ENSG00000112715,ENSG00000074266,...,ENSG00000106263,ENSG00000065883,ENSG00000039068,ENSG00000145041,ENSG00000171681,ENSG00000079999,ENSG00000168036,ENSG00000166949,ENSG00000136997,ENSG00000118058
2,ENSG00000055130,ENSG00000051180,ENSG00000087586,ENSG00000177885,ENSG00000103266,ENSG00000168036,ENSG00000065361,ENSG00000174842,ENSG00000129946,ENSG00000178691,...,ENSG00000101076,ENSG00000136807,ENSG00000004975,ENSG00000127129,ENSG00000141644,ENSG00000005339,ENSG00000204209,ENSG00000175387,ENSG00000059728,ENSG00000130522
3,ENSG00000105173,ENSG00000163939,ENSG00000196591,ENSG00000050820,ENSG00000116478,ENSG00000134982,ENSG00000080824,ENSG00000177885,ENSG00000099250,ENSG00000112511,...,ENSG00000091831,ENSG00000182944,ENSG00000147140,ENSG00000112619,ENSG00000142208,ENSG00000197063,ENSG00000177374,ENSG00000157933,ENSG00000134323,ENSG00000129691
4,ENSG00000136997,ENSG00000127616,ENSG00000137337,ENSG00000091732,ENSG00000161405,ENSG00000070018,ENSG00000112851,ENSG00000146648,ENSG00000166167,ENSG00000008083,...,ENSG00000131323,ENSG00000090061,ENSG00000131508,ENSG00000164610,ENSG00000156299,ENSG00000171617,ENSG00000143799,ENSG00000170365,ENSG00000119950,ENSG00000117222


# Training

In [ ]:
for GENE in Gene_N.columns:    
    gene_100=Gene_N[GENE]
    gene_100= [x for x in Gene_N[GENE]  if x == x]
    TCGA_MTX=Normalization.MTX(Df, gene_100)
    TCGA_MTX=TCGA_MTX.T
    filename = "/data2/sano/TCGA_Version_LAST/Models/MTX_"+GENE+".csv"
    TCGA_MTX.to_csv(filename )
        
    train_x =  TCGA_MTX.loc[Train_Test_Patients["train"]]
    train_y = Driver_Mutation_Matrix[GENE].loc[Train_Test_Patients["train"]]

    clf1 = RandomForestClassifier(max_depth=5, n_estimators=10000, random_state=0, class_weight='balanced')
    clf1.fit(train_x, train_y)

    filename = "/data2/sano/TCGA_Version_LAST/Models/Model_"+GENE+".sav"
    with open(filename, 'wb') as f:
        pickle.dump(clf1, f)

# Calculate SLP score

In [55]:
GENE="TP53"
filename1 = "/data2/sano/TCGA_Version_LAST/Models/Model_"+GENE+".sav"
filename2 = "/data2/sano/TCGA_Version_LAST/Models/MTX_"+GENE+".csv"
gene_100=Gene_N[GENE]
with open(filename1, 'rb') as f:
    clf=pickle.load(f)
TCGA_MTX=Mut_pred.MTX(Df[["TCGA-26-5134-01"]], gene_100).T
SLPs=pd.DataFrame(clf.predict_proba(TCGA_MTX)).iloc[0,0]

In [73]:
SLPs

0.6996237398766825